# Ray RLlib - RecSys: Recommender System

© 2019-2022, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademyLogo.png)

This section explores one approach for using *reinforcement learning* with [Ray RLlib](https://rllib.io/) to build a [*recommender system*](https://en.wikipedia.org/wiki/Recommender_system).

For reference, the GitHub public repo for this code is available at <https://github.com/anyscale/academy/blob/main/ray-rllib/recsys> and full source code for this example recommender system is also in the `recsys.py` script. You can run that with default settings to exercise the code:

```shell
python recsys.py
```

To see the available command line options use:

```shell
python recsys.py --help
```

A full run takes about 5-10 minutes on a recent model MacBook Pro laptop.

## Load the Training Data

The approach given here for building a recommender system could be applied to just about any dataset where *users* are *rating* a set of *items*. It's bounded in terms of memory requirements as the number of users grows. It could be re-engineered to handle a very large set of items.

We'll be using the [Jester collaborative filtering dataset](https://goldberg.berkeley.edu/jester-data/) – which is known for having a high *density*, i.e., where many users have rated many of the items: https://goldberg.berkeley.edu/jester-data/.

Jester was an online joke recommender hosted a UC Berkeley which collected data from April 1999 through May 2003. See the discussion of "universal queries" in:

> "Eigentaste: A Constant Time Collaborative Filtering Algorithm"  
Ken Goldberg, Theresa Roeder, Dhruv Gupta, Chris Perkins  
*Information Retrieval*, 4(2), 133-151 (July 2001)  
<https://goldberg.berkeley.edu/pubs/eigentaste.pdf>

The data is split into three downloadable files, and the first file contains anonymous ratings from 24,983 users who have rated 36 or more jokes.

Ratings data is organized as a matrix with dimensions `24983 X 101`

  * one row per user
  * first column gives the number of jokes rated by that user
  * the next 100 columns give the ratings for jokes `01` through `100`
  * ratings are real values ranging from `-10.00` to `+10.00`
  * the value `"99"` corresponds to `None` = "not rated"
  
Here's a function to load the dataset:

In [ ]:
import csv

NO_RATING = "99"
MAX_RATING = 10.0

def load_data (data_path):
    rows = []

    with open(data_path, newline="") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=",")

        for row in csvreader:
            conv = [None] * (len(row) - 1)

            for i in range(1, len(row)):
                if row[i] != NO_RATING:
                    rating = float(row[i]) / MAX_RATING
                    conv[i - 1] = rating

            rows.append(conv)

    return rows

## Data Discovery

Let's load the dataset into a `pandas` dataframe, then take a look at it:

In [ ]:
from pathlib import Path
import os
import pandas as pd

DATA_PATH = Path(os.getcwd()) / Path("jester-data-1.csv")
sample = load_data(DATA_PATH)

df = pd.DataFrame(sample)
df.head()

Notice above how there are many `NaN` values in the dataset.
Those are *missing values*.
Before we analyze this data any further, we'll need to [*impute*](https://en.wikipedia.org/wiki/Imputation_(statistics)) those missing values.
In other words, substitute each missing value with some appropriate estimate, such as a column mean.
The `SimpleImputer` class in `scikit-learn` provides a good way to handle that requirement:
<https://scikit-learn.org/stable/modules/impute.html>.

We'll save the imputed sample data in the `X` matrix:

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy="mean")
imp.fit(df.values)

X = imp.transform(df.values).T

print(X)

Next, we'll use *K-means clustering* as an unsupervised learning approach to understand more about how the dataset is structured. In other words, let's re-arrange the dataset into `k` clusters of items, based on how many different users have rated them.

Since `k` is a *hyperparameter* we'll need some way to select an appropriate value for it. One approach that's popular for K-means is to evaluate the *inertia* of decreasing error in the clustering models as `k` grows from zero to some limit.

 * <https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_stability_low_dim_dense.html>
 * <https://towardsdatascience.com/k-means-clustering-with-scikit-learn-6b47a369a83c>
 
We really don't want to work with more than a few clusters – since many small clusters would lead to less *predictive power* in the _recsys_. Given that there are 100 items, let's measure inertia up to `k=20` clusters.

In [ ]:
from sklearn.cluster import KMeans

distortions = []

for i in range(1, 20):
    km = KMeans(
        n_clusters=i,
        init="random",
        n_init=10,
        max_iter=300,
        tol=1e-04,
        random_state=0
    )

    km.fit(X)
    distortions.append(km.inertia_)

Let's visualize the curve of inertia, using `matplotlib`:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(range(1, 20), distortions, marker="o")
plt.xlabel("Number of clusters")
plt.ylabel("Distortion")
plt.show()

This kind of cluster analysis has stochastic aspects, so results may differ on different runs. Generally, the plot shows a "knee" in the curve near `k=7` as the decrease in error begins to level out. That's a reasonable number of clusters, such that each cluster will tend to have ~14% of the items. That choice has an inherent trade-off:

  * too few clusters → poor predictions (less precision)
  * too many clusters → poor predictive power (less recall)

Now we can run K-means in `scikit-learn` with that hyperparameter `k=7` to get the clusters that we'll use in our RL environment:

In [ ]:
K_CLUSTERS = 7

km = KMeans(n_clusters=K_CLUSTERS)
km.fit(X)

y_km = km.fit_predict(X)

From this analysis we'll need the *labels* for each item, i.e., the number of the cluster to which each item gets assigned:

In [ ]:
from collections import defaultdict

labels = km.labels_
clusters = defaultdict(set)

for i in range(len(labels)):
    clusters[labels[i]].add(i)

CLUSTERS = dict(clusters)

We'll also use the cluster *centers*, which provide the vector positions (the per-dimension "means") for each cluster.

In [ ]:
centers = km.cluster_centers_
CENTERS = centers.tolist()

BTW, let's take a look at the K clusters from this analysis…

In [ ]:
plt.scatter(
    X[y_km == 0, 0], X[y_km == 0, 1],
    s=50, c="lightgreen",
    marker="s", edgecolor="black",
    label="cluster 0"
)

plt.scatter(
    X[y_km == 1, 0], X[y_km == 1, 1],
    s=50, c="orange",
    marker="o", edgecolor="black",
    label="cluster 1"
)

plt.scatter(
    X[y_km == 2, 0], X[y_km == 2, 1],
    s=50, c="lightblue",
    marker="v", edgecolor="black",
    label="cluster 2"
)

plt.scatter(
    X[y_km == 3, 0], X[y_km == 3, 1],
    s=50, c="blue",
    marker="^", edgecolor="black",
    label="cluster 3"
)

plt.scatter(
    X[y_km == 4, 0], X[y_km == 4, 1],
    s=50, c="yellow",
    marker="<", edgecolor="black",
    label="cluster 4"
)

plt.scatter(
    X[y_km == 5, 0], X[y_km == 5, 1],
    s=50, c="purple",
    marker=">", edgecolor="black",
    label="cluster 5"
)

plt.scatter(
    X[y_km == 6, 0], X[y_km == 6, 1],
    s=50, c="brown",
    marker="X", edgecolor="black",
    label="cluster 6"
)

# plot the centroids
plt.scatter(
    km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
    s=250, marker="*",
    c="red", edgecolor="black",
    label="centers"
)

plt.legend(scatterpoints=1)
plt.grid()
plt.show()

Not bad, based on the centers these clusters show some separation – at least when we plot in 2 dimensions.

## Thesis for Building an RL Recommender System

We're using a thesis here that people tend to like similar "kinds" of items, in this case jokes. While we could go to extra effort to build a classifier model for jokes, it's much simpler to measure ratings for them across a large sample of users' likes. In a classic [*collaborative filtering*](https://en.wikipedia.org/wiki/Collaborative_filtering) approach, we could base our recommendations on other users who have similar patterns of rating items. 

On the one hand, that may be expensive to scale as the number of users grows. Instead, an [*item-based collaborative filter*](https://en.wikipedia.org/wiki/Item-item_collaborative_filtering) approach which was invented by [Amazon](https://ieeexplore.ieee.org/document/1167344) for "people who bought x also bought y". This scales to massive datasets by clustering the items based on user ratings.

On the other hand, people change their interests over time but a class CF model doesn't account for that.  For a thorough study of these effects, see ["From Amateurs to Connoisseurs: Modeling the Evolution of User Expertise through Online Reviews"](https://arxiv.org/abs/1303.4402) by Julian McAuley and Jure Leskovec (2013-03-18).

Let's use reinforcement learning to build an item-based recommender system that can "evolve" its recommendations for each individual over time. We'll leverage *exploration* (gather info that leads toward better recommendations in the future) versus *exploitation* (use the best decision given the info known so far). We have clustered the items (jokes), so now we can create a custom environment where:

  * *actions* taken by an agent – recommend items from specific clusters
  * *observation space* – a vector distance from the user's rating history to the cluster centers
  * *rewards* – user ratings for recommended items, with minor penalties for unrated items
  
Each *episode* for an agent will simulate one user's ratings vs. which jokes get recommdened based on our *policy* which is trained using RL.

One other point: a *dense sub-matrix* – in which almost all users have rated the jokes – is defined by the following columns:

In [ ]:
DENSE_SUBMATRIX = [ 5, 7, 8, 13, 15, 16, 17, 18, 19, 20 ]

One well-known issue encountered in production use of recommenders is the [*cold start*](https://en.wikipedia.org/wiki/Cold_start_(recommender_systems)) problem. When we first begin to make recommendations for a given user, we have zero information about them. So it's not possible to provide personalized recommendations. However, we have the dense sub-matrix above, which almost all users have rated. We can sample from this subset – as "warm" items – to provide to each user at the start of an *episode*. That will give the recommender some initial information to leverage as a starting point.

## A Custom Gym Environment

Next we'll build a custom environment for training a policy with RLlib.  We'll use OpenAI [Gym](https://gym.openai.com/) as the basis for it. For details about how to build custom Gym environments for working with RLlib, see ["Anatomy of a custom environment for RLlib"](https://medium.com/distributed-computing-with-ray/anatomy-of-a-custom-environment-for-rllib-327157f269e5) and its accompanying source code in the <https://github.com/DerwenAI/gym_example> Git repo. Some details are also covered in the _Explore RLlib_ lesson, [Custom Environments and Reward Shaping](../explore-rllib/03-Custom-Environments-Reward-Shaping.ipynb)

First, we'll define a few globals… `MAX_STEPS` is the length of an episode, based on how many jokes can be recommended before repeating. Reward values are either the user's rating for the recommended item, or minor penalties for recommending either a "depleted" cluster or an item which the user never rated.

In [ ]:
ROW_LENGTH = 100
MAX_STEPS = ROW_LENGTH - len(DENSE_SUBMATRIX)

REWARD_DEPLETED = -0.1	# item recommended from a depleted cluster (no-op)
REWARD_UNRATED = -0.05	# item was never rated by this user

Now for the Gym environment. This is one class and should be defined within one cell – albeit this is a long cell to scroll through…

In [ ]:
import gym
from gym import spaces
import random

class JokeRec (gym.Env):
    def __init__ (self, config):
        # NB: here we're passing strings via config; RLlib use of JSON
        # parser was throwing exceptions due to config values
        self.dense = eval(config["dense"])
        self.centers = eval(config["centers"])
        self.clusters = eval(config["clusters"])

        lo = np.array([np.float64(-1.0)] * K_CLUSTERS)
        hi = np.array([np.float64(1.0)] * K_CLUSTERS)

        self.observation_space = spaces.Box(lo, hi, shape=(K_CLUSTERS,), dtype=np.float64)
        self.action_space = spaces.Discrete(K_CLUSTERS)

        # load the dataset
        self.dataset = load_data(config["dataset"])


    def _warm_start (self):
        """
        attempt a warm start, sampling half the dense submatrix of most-rated items
        """
        sample_size = round(len(self.dense) / 2.0)

        for action, items in self.clusters.items():
            for item in random.sample(self.dense, sample_size):
                if item in items:
                    state, reward, done, info = self.step(action)


    def _get_state (self):
        """
        calculate root-mean-square (i.e., normalized vector distance) for the agent's current 
        "distance" measure from each cluster center as the observation
        """
        n = float(len(self.used))

        if n > 0.0:
            state = [ np.sqrt(x / n) for x in self.coords ]
        else:
            state = self.coords

        return state


    def reset (self):
        """
        reset the item recommendation history, select a new user to simulate from among the 
        dataset rows, then run an initial 'warm-start' sequence of steps before handing step
        control back to the agent
        """
        self.count = 0
        self.used = []
        self.depleted = 0
        self.coords = [np.float64(0.0)] * K_CLUSTERS

        # select a random user to simulate
        self.data_row = random.choice(self.dataset)

        # attempt a warm start
        self._warm_start()

        return self._get_state()


    def step (self, action):
        """
        recommend one item, which may result in a no-op -- 
        in production, skip any repeated items per user
        """
        assert action in self.action_space, action
        assert_info = "c[item] {}, rating {}, scaled_diff {}"

        # enumerate items from the cluster selected by the action that haven't been recommended
        # previously to the simulated user
        items = set(self.clusters[action]).difference(set(self.used))

        if len(items) < 1:
            # oops! items from the selected cluster have been depleted, i.e. all have been 
            # recommended previously to the simulated user; hopefully the agent will learn
            # to switch to exploring among the other clusters
            self.depleted += 1
            item = None
            reward = REWARD_DEPLETED
        else:
            # chose an item at random from the selected cluster
            item = random.choice(list(items))
            rating = self.data_row[item]

            if not rating:
                # no-op! this action resulted in an unrated item
                reward = REWARD_UNRATED

            else:
                # success! this action resulted in an item rated by the simulated user
                reward = rating
                self.used.append(item)

                # update the coords history: agent observes its distance to each cluster "evolve"
                for i in range(len(self.coords)):
                    c = self.centers[i]
                    scaled_diff = abs(c[item] - rating) / 2.0
                    self.coords[i] += scaled_diff ** 2.0

        self.count += 1
        done = self.count >= MAX_STEPS
        info = { "item": item, "count": self.count, "depleted": self.depleted }

        return self._get_state(), reward, done, info


    def render (self, mode="human"):
        last_used = self.used[-10:]
        last_used.reverse()
        print(">> used:", last_used)
        print(">> dist:", [round(x, 2) for x in self._get_state()])
        print(">> depl:", self.depleted)

The `JokeRec.step()` method contains the heart of the simulation logic here. It determines the item to be recommend based on the input `action`, then determines the `reward` and the agent's updated vector distance to the cluster centers as the `observation space`.

Overall, this approach is relatively well-behaved and bounded for its memory use as the number of items grows. We're keeping most of the data in memory, but could readily use a distributed key/value store for selecting items, maintaining user history, etc.

## Configuration

Now we'll use the results of K-means clustering on the data sample to prepare a configuration for our custom environment.

We'll use [*proximal policy optimization*](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=ppo#proximal-policy-optimization-ppo) (PPO) for training a policy in RLlib, taking the default PPO configuration as the foundation here.

In [ ]:
import ray
import ray.rllib.agents.ppo as ppo

CONFIG = ppo.DEFAULT_CONFIG.copy()

CONFIG["log_level"] = "WARN"
CONFIG["num_workers"] = 3    # set to `0` for debug

CONFIG["env_config"] = {
    "dataset": DATA_PATH,
    "dense": str(DENSE_SUBMATRIX),
    "clusters": repr(CLUSTERS),
    "centers": repr(CENTERS),
}

Now let's create an instance of our custom Gym environment and call the `reset()` method, which will initialize an episode (simulating one user's ratings), run a "warm start", then return the initial observation:

In [ ]:
env = JokeRec(CONFIG["env_config"])
env.reset()

We can randomly select an action, i.e., the label of an item cluster to recommend:

In [ ]:
action = env.action_space.sample()
print("action:", action)

Let's use that action to take one step:

In [ ]:
state, reward, done, info = env.step(action)
print("obs:", state)
print("reward:", reward)

We can call the `render()` method to describe more about the agent's state within its environment:

In [ ]:
env.render()

Notice how there are already some other "used" items, due to the warm start.

Next, we'll define a function that runs through one entire episode:

In [ ]:
def run_one_episode (env, naive=False, verbose=False):
    """
    step through one episode, using either a naive strategy or random actions
    """
    env.reset()
    sum_reward = 0

    action = None
    avoid_actions = set([])
    depleted = 0

    for i in range(MAX_STEPS):
        if not naive or not action:
            action = env.action_space.sample()

        state, reward, done, info = env.step(action)

        if verbose:
            print("action:", action)
            print("obs:", i, state, reward, done, info)

        # naive strategy: select items from the nearest non-depleted cluster
        if naive:
            if info["depleted"] > depleted:
                depleted = info["depleted"]
                avoid_actions.add(action)

            obs = []

            for a in range(len(state)):
                if a not in avoid_actions:
                    dist = round(state[a], 2)
                    obs.append([dist, a])

            action = min(obs)[1]

        sum_reward += reward

        if done:
            if verbose:
                print("DONE @ step {}".format(i))

            break

    if verbose:
        print("CUMULATIVE REWARD: ", round(sum_reward, 3))

    return sum_reward

Next, another function which uses `run_one_episode()` to measure the baseline performance of a "naïve" strategy, i.e., without use of reinforcement learning to train a policy:

In [ ]:
from tqdm import tqdm

def measure_baseline (env, n_iter=1, naive=False, verbose=False):
    history = []

    for episode in tqdm(range(n_iter), ascii=True, desc="measure baseline"):
        sum_reward = run_one_episode(env, naive=naive, verbose=verbose)
        history.append(sum_reward)

    baseline = sum(history) / len(history)
    return baseline

Now let's use this to measure how well our recommender system runs without leveraging RLlib:

In [ ]:
baseline = measure_baseline(env, n_iter=1000, naive=True)
print("BASELINE CUMULATIVE REWARD", round(baseline, 3))

## Running Ray and RLlib

At this point we're ready to train a policy with RLlib. First we'll initialize the directory in which to save *checkpoints*, and the directory in which to log results…

In [ ]:
import os
import shutil

CHECKPOINT_ROOT = "tmp/rec"
shutil.rmtree(CHECKPOINT_ROOT, ignore_errors=True, onerror=None)

ray_results = "{}/ray_results/".format(os.getenv("HOME"))
shutil.rmtree(ray_results, ignore_errors=True, onerror=None)

Now start Ray, register our custom environment, and create an agent. BTW, if you see lots of "deprecation" warnings from [Tensorflow](https://www.tensorflow.org/) just ignore those…

In [ ]:
info = ray.init(ignore_reinit_error=True)

Register our environment so we can reference it by name. Then create a `PPOTrainer`.

In [ ]:
from ray.tune.registry import register_env

env_key = "JokeRec-v0"
register_env(env_key, lambda config_env: JokeRec(config_env))
AGENT = ppo.PPOTrainer(CONFIG, env=env_key)

Let's train a policy using the PPO optimizer in RLlib. As the following code steps through each training iteration, watch how the measured improvements in the min, mean, and max rewards per episode.

In [ ]:
TRAIN_ITER = 20

df = pd.DataFrame(columns=[ "min_reward", "avg_reward", "max_reward", "steps", "checkpoint"])
status = "reward {:6.2f} {:6.2f} {:6.2f}  len {:4.2f}  saved {}"

for i in range(TRAIN_ITER):
    result = AGENT.train()
    checkpoint_file = AGENT.save(CHECKPOINT_ROOT)

    row = [
        result["episode_reward_min"],
        result["episode_reward_mean"],
        result["episode_reward_max"],
        result["episode_len_mean"],
        checkpoint_file,
        ]

    df.loc[len(df)] = row
    print(status.format(*row))
    
BEST_CHECKPOINT = checkpoint_file

## Rollout to Emulate a Use Case Deployment

Now let's define a function to run a *rollout* using a checkpointed policy.
Each rollout iteration will emuluate a deployed use of our recommender system for one user, and we'll measure the average rewards across many iterations:

In [ ]:
def run_rollout (agent, env, n_iter=1, verbose=False):
    """
    iterate through `n_iter` episodes in a rollout to emulate deployment in a production use case
    """
    for episode in range(n_iter):
        state = env.reset()
        sum_reward = 0

        for step in range(MAX_STEPS):
            try:
                action = agent.compute_action(state)
                state, reward, done, info = env.step(action)
                sum_reward += reward

                if verbose:
                    print("reward {:6.3f}  sum {:6.3f}".format(reward, sum_reward))
                    env.render()
            except Exception:
                traceback.print_exc()

            if done:
                # report at the end of each episode
                print("CUMULATIVE REWARD:", round(sum_reward, 3), "\n")
                yield sum_reward
                break

Now we can apply the best trained policy in a rollout:

In [ ]:
AGENT.restore(BEST_CHECKPOINT)
history = []

for episode_reward in run_rollout(AGENT, env, n_iter=500, verbose=False):
    history.append(episode_reward)
    
print("average reward:", round(sum(history) / len(history), 3))

How does the reported *average reward* from many rollouts (using RLlib to train a policy) compare with the *baseline cumulative reward* above based on a naïve strategy and no learning?  How does it compare with the predicted *mean reward per episode* from training? 

The baseline reward from a naïve strategy should be much lower (worse user ratings) than the other two measures.

These measures are an estimate for how a user would rate their recommended items.
Of course, not all users will like the jokes, so there will be some rollouts with negative rewards.
Overall we want the average reward to be *positive*, with `MAX_STEPS` as an upper bounds.

## Examine the trained policy

Use the following code to examine the trained policy that was optimized using PPO:

In [ ]:
policy = AGENT.get_policy()
model = policy.model

print("\n", model.base_model.summary())

## Evaluate learning with TensorBoard

You also can run [TensorBoard](https://www.tensorflow.org/tensorboard) to visualize the RL training metrics from the log files. The results during training were written to a directory under `$HOME/ray_results`

If you are viewing this lesson on the Anyscale hosted platform, use the provided link to open TensorBoard.

If you are viewing this lesson on a laptop, open a terminal and run the following command, then open the URL shown in the output. (You can open a terminal using the `+` in the upper left-hand corner of Jupyter Lab.)

```shell
tensorboard --logdir=~/ray_results
```

Open the URL printed to view the TensorBoard GUI.

---

## Exercise 1

Compare use of the other datasets `"jester-data-2.csv"` and `"jester-data-3.csv"` by substituting them during the rollout.

How do the mean cumulative reward differ from the metrics in the lesson?

## Exercise 2

Compare the effect of using a larger `K` value for the number of clusters.

Show the difference, if any, by comparing:

  * baseline with random actions 
  * baseline with the naïve strategy
  * predicted average reward from training
  * stats from the rollout

## Discussion Questions

  1. In what ways could the "warm start" be improved?
  2. How could this code be modified to scale to millions of users?  Or to thousands of items?

## Clean up

Finally, let's shutdown Ray gracefully:

In [ ]:
ray.shutdown()